In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [48]:
def get_pop_data():
    remove_words = ['County', 'Municipality', 'Parish', ]
    county_pop = pd.read_csv('./../county_pop.csv')
    county_pop = county_pop[['STNAME', 'CTYNAME', 'CENSUS2010POP', 'COUNTY']]
    county_pop.columns = ['state', 'county', 'pop', 'county_num']
    for word in remove_words:
        county_pop.county = county_pop.county.apply(lambda x: x.replace(f' {word}', '').strip())
        
    # add custom pops
    custom_pops = pd.DataFrame({
        'state': ['Missouri', 'New York', 'Missouri', 'New Mexico'],
        'county': ['Kansas City', 'New York City', 'Joplin', 'Doña Ana'],
        'pop': [491918, 8399000, 50657, 218195],
        'county_num': [99 for i in range(4)]
    })
    return pd.concat([county_pop, custom_pops])

def get_covid_data():
    return pd.read_csv('./../us-counties.csv')

def get_county_data():
    county_covid = get_covid_data()
    county_pop = get_pop_data()
    county = county_covid.set_index(
        ['state', 'county']
    ).join(
        county_pop.set_index(['state', 'county'])
    )
    county = county.drop(columns=['county_num'])
    return county

def add_per_capita(df):
    df = df.copy()
    df['cases_per_capita'] = df['cases'] / df['pop']
    df['deaths_per_capita'] = df['deaths'] / df['pop']
    df['cases_per_k'] = df['cases_per_capita'] * 1000.0
    df['deaths_per_k'] = df['deaths_per_capita'] * 1000.0
    return df

def add_pct_changes(df, func, cols, new_names):
    chg = df.copy()
    chg = getattr(chg[cols], func)()
    chg.columns = new_names
    for col in new_names:
        chg[chg[col] <= -0.998] = np.NaN
    return df.join(chg)
        
def add_day_since_outbreak_numbers(df):
    df = df.copy()
    df['day_since_first_case'] = list(
        df
        .reset_index()
        .groupby(df.index.names[:-1])
        .cumcount()
    )
    return df


def get_data():
    county = get_county_data()
    pop = get_pop_data()
    
    state = county[['date', 'cases', 'deaths']].groupby(['state', 'date']).sum()
    state = state.join(pop[pop.county_num == 0].set_index('state'))
    state = state.drop(columns=['county_num', 'county'])
    
    country = county[['date', 'cases', 'deaths']].groupby(['date']).sum()
    country['pop'] = 328200000.0
    country['Country'] = 'United States'
    country = country.reset_index().set_index(['Country', 'date'])
    
    county = (
        county
        .reset_index()
        .set_index(['state', 'county', 'date'])
    )
    
    dfs = [country, state, county]
    dfs = [add_per_capita(df) for df in dfs]
    dfs = [
        add_pct_changes(
            df, 
            'pct_change',
            ['cases', 'deaths'], 
            ['cases_pct_change', 'death_pct_change']
        ) for df in dfs
    ]
    dfs = [
        add_pct_changes(
            df, 
            'pct_change',
            ['cases_pct_change', 'death_pct_change'], 
            ['cases_pct_change_pct_change', 'death_pct_change_pct_change']
        ) for df in dfs
    ]
    dfs = [
        add_pct_changes(
            df, 
            'diff',
            ['cases', 'deaths'], 
            ['new_cases', 'new_deaths']
        ) for df in dfs
    ]
    dfs = [add_day_since_outbreak_numbers(df) for df in dfs]
    
    return dfs
    


In [49]:
country, state, county = get_data()


In [50]:
country

cases  deaths          pop  cases_per_capita  \
Country       date                                                         
United States 2020-01-21        1       0  328200000.0      3.046923e-09   
              2020-01-22        1       0  328200000.0      3.046923e-09   
              2020-01-23        1       0  328200000.0      3.046923e-09   
              2020-01-24        2       0  328200000.0      6.093845e-09   
              2020-01-25        3       0  328200000.0      9.140768e-09   
              2020-01-26        5       0  328200000.0      1.523461e-08   
              2020-01-27        5       0  328200000.0      1.523461e-08   
              2020-01-28        5       0  328200000.0      1.523461e-08   
              2020-01-29        5       0  328200000.0      1.523461e-08   
              2020-01-30        6       0  328200000.0      1.828154e-08   
              2020-01-31        7       0  328200000.0      2.132846e-08   
              2020-02-01        8       0  328200000.0      2.437538e-08   
              2020-02-02       11       0  328200000.0      3.351615e-08   
              2020-02-03       11       0  328200000.0      3.351615e-08   
              2020-02-04       11       0  328200000.0      3.351615e-08   
              2020-02-05       12       0  328200000.0      3.656307e-08   
              2020-02-06       12       0  328200000.0      3.656307e-08   
              2020-02-07       12       0  328200000.0      3.656307e-08   
              2020-02-08       12       0  328200000.0      3.656307e-08   
              2020-02-09       12       0  328200000.0      3.656307e-08   
              2020-02-10       13       0  328200000.0      3.960999e-08   
              2020-02-11       13       0  328200000.0      3.960999e-08   
              2020-02-12       14       0  328200000.0      4.265692e-08   
              2020-02-13       15       0  328200000.0      4.570384e-08   
              2020-02-14       15       0  328200000.0      4.570384e-08   
              2020-02-15       15       0  328200000.0      4.570384e-08   
              2020-02-16       15       0  328200000.0      4.570384e-08   
              2020-02-17       25       0  328200000.0      7.617307e-08   
              2020-02-18       25       0  328200000.0      7.617307e-08   
              2020-02-19       25       0  328200000.0      7.617307e-08   
              2020-02-20       27       0  328200000.0      8.226691e-08   
              2020-02-21       30       0  328200000.0      9.140768e-08   
              2020-02-22       30       0  328200000.0      9.140768e-08   
              2020-02-23       30       0  328200000.0      9.140768e-08   
              2020-02-24       43       0  328200000.0      1.310177e-07   
              2020-02-25       45       0  328200000.0      1.371115e-07   
              2020-02-26       60       0  328200000.0      1.828154e-07   
              2020-02-27       60       0  328200000.0      1.828154e-07   
              2020-02-28       65       0  328200000.0      1.980500e-07   
              2020-02-29       70       1  328200000.0      2.132846e-07   
              2020-03-01       88       3  328200000.0      2.681292e-07   
              2020-03-02      104       6  328200000.0      3.168800e-07   
              2020-03-03      125      10  328200000.0      3.808653e-07   
              2020-03-04      161      12  328200000.0      4.905545e-07   
              2020-03-05      228      12  328200000.0      6.946984e-07   
              2020-03-06      311      15  328200000.0      9.475929e-07   
              2020-03-07      429      19  328200000.0      1.307130e-06   
              2020-03-08      548      22  328200000.0      1.669714e-06   
              2020-03-09      752      26  328200000.0      2.291286e-06   
              2020-03-10     1022      31  328200000.0      3.113955e-06   
              2020-03-11     1273      37  328200000.0      3.878732e-06   


In [51]:
# Create traces
states_im_interested_in = ['California', 'Idaho', 'Utah', 'New York', 'Montana', 'Texas']
df = state.reset_index()
df = df[[state in states_im_interested_in for state in df.state]]
cols = ['cases', 'cases_per_k', 'new_cases', 'cases_pct_change', 'cases_pct_change_pct_change']
for col in cols:
    fig = go.Figure()
    for s in df.state.unique():
        fig.add_trace(go.Scatter(x=df[df.state == s].day_since_first_case, y=df[df.state == s][col],
                            mode='lines+markers',
                            name=s))

    fig.show()
